In [0]:
tweets = [ "Fiesta en la playa #BeachParty#SunFun#CacaSun, bronceado extremo!",
"#CaféEnLaMañana con un libro... #Relax pero mi café está frío :(",
"#CaféEnLaMañana con un libro... #Relax pero mi café está frío :(",
"#CacaSun con un libro... #Relax pero mi café está frío :(",
"#CaféEnLaMañana con un libro... #Relax pero mi café está frío :(",
"Noche de estudio #StudyHard#NoSleep, ¿quién inventó los exámenes?",
"#CacaSun#CacaSun, corriendo bajo la lluvia... totalmente empapado",
"Pizza con amigos, #CacaSun#CacaSun pero ¿quién tomó mi trozo?",
"Perdido en la ciudad #CityLife#Lost pero descubriendo lugares nuevos",
"#GymTime#Workout, pero realmente odio las sentadillas" ]

In [0]:
PALABRAS_PROHIBIDAS             = ("caca", "culo", "pedo", "pis")
sc.broadcast(PALABRAS_PROHIBIDAS) # Mando la lista de palabras prohibidas una única vez a los worksers.. que la mantienen en memoria, durante todo el trabajo.
NUMERO_RESULTADOS_A_DEVOLVER    = 5

In [0]:
hashtags_eliminados = 0
# Un broadcast es una variable de SOLO LECTURA que mando a los workers... y la mantienen en memoria
# Un accumulator es una variablke COMPARTIDA para escritura entre todos los workers... y el master.

def contiene_palabra_prohibida(palabra):
    global hashtags_eliminados
    for palabra_prohibida in PALABRAS_PROHIBIDAS:
        if palabra_prohibida in palabra:
            hashtags_eliminados += 1
            print("QUITO UNA")
            return False
    return True

print(all( [True, True, True]))
print(all( [True, True, False]))
print(all( [False, True, False]))
print(all( [False, False, False]))

funcion = lambda palabra: not any([palabra_prohibida in palabra for palabra_prohibida in PALABRAS_PROHIBIDAS ])
funcion("manculozana")

True
False
False
False


False

In [0]:
import re

rdd = sc.parallelize(tweets) 
rdd = rdd.map(         lambda tweet: tweet.replace("#", " #")                  ) 
rdd = rdd.flatMap(     lambda tweet: re.split(r'[ .,;:;_()!¿¡-]', tweet)       )    \
            .filter(      lambda token: token.startswith("#")                     ) \
            .map(         lambda hashtag: hashtag[1:]                             ) \
            .map(         lambda hashtag: hashtag.lower()                         ) \
            .filter(      contiene_palabra_prohibida                              ) \
            .map(         lambda hashtag: (hashtag,1)                             ) \
            .reduceByKey( lambda numero1, numero2: numero1 + numero2              ) \
            .sortBy(      lambda hashtag_numero_apariciones: hashtag_numero_apariciones[1] , ascending=False ) \
            .take(        NUMERO_RESULTADOS_A_DEVOLVER                            )
#            .filter(      lambda hashtag: not any([palabra_prohibida in hashtag for palabra_prohibida in PALABRAS_PROHIBIDAS ])   ) \

print(rdd)

[('relax', 4), ('caféenlamañana', 3), ('beachparty', 1), ('nosleep', 1), ('gymtime', 1)]


In [0]:
print(hashtags_eliminados)

0
